In [1]:
# Importing packages

import pandas as pd
import numpy  as np
import seaborn as sns
import datetime 
import chardet


# Goal 1: Automating Pollutant Assessment 
Current Situation: 
- ​To evaluate a material's reusability, users must manually input information about 
pollutants 
- ​These pollutants (adhesives, paints, fire retardants, etc.) affect how easily materials can 
be separated and reused 
- ​The manual nature of this process introduces inconsistency and inefficiency 



## Objective: Develop a system that automatically suggests strong default pollutant values based on the materials being assessed. 
This system should: 
- ​Predict likely pollutants based on material combinations 
- Provide appropriate pollutant classifications 
- Reduce manual input while improving assessment accuracy 

## Methodology and approach
- Machine learning models to predict pollutants based on material combinations 
- Pattern recognition systems that identify common construction methods and associated 
pollutants 
- Rule-based systems derived from expert knowledge 
- ​Default value frameworks with adjustment mechanisms based on material properties 

## Guiding Questions

1. How can we identify which materials are commonly used together in building components?
2. What properties in the datasets can help us determine the type of connections used between materials?
3. How might different connection types affect the end-of-life scenarios in the tBaustoff dataset?
4. Can we classify connections based on their environmental impact using the lifecycle assessment data?
5. What metrics could we use to define the "disturbance potential" of different connection types?
6. How can we develop a recommendation system for material connections that optimizes for circularity?

# Data Wrangling

## Data Extraction

In [7]:
# detect files encoding
with open("/Users/pablosoriano/Documents/Data Science/bbsr-challenge/csv/OBD_2024_I.csv", "rb") as f:
    result = chardet.detect(f.read(100000))  # Read first 100,000 bytes
    print(result)

{'encoding': 'ISO-8859-1', 'confidence': 0.7295400999999999, 'language': ''}


In [ ]:
# Importing data from CSV and converting to dataframe
oko_2024_df = pd.read_csv("/Users/pablosoriano/Documents/Data Science/bbsr-challenge/csv/OBD_2024_I.csv", delimiter=";", encoding = result["encoding"], low_memory=False)
oko_2023_df= pd.read_csv("/Users/pablosoriano/Documents/Data Science/bbsr-challenge/csv/OBD_2023_I.csv", delimiter=";", encoding = result["encoding"], low_memory=False)
oko_2020_df = pd.read_csv("/Users/pablosoriano/Documents/Data Science/bbsr-challenge/csv/OBD_2020_II.csv", delimiter=";", encoding = result["encoding"], low_memory=False)
tbau_df = pd.read_csv("/Users/pablosoriano/Documents/Data Science/bbsr-challenge/csv/tBaustoff_with_OBD_mapping.csv", delimiter=";", encoding = result["encoding"], low_memory=False)


In [ ]:
# detect files encoding for pollutant_combinations.csv
with open("/Users/pablosoriano/Documents/Data Science/bbsr-challenge/csv/pollutant_combinations.csv", "rb") as f:
    result = chardet.detect(f.read(100000))  # Read first 100,000 bytes
    print(result)

{'encoding': 'utf-8', 'confidence': 0.99, 'language': ''}


In [ ]:
# importing pullutant_combinations.csv
pollutants_df = pd.read_csv("/Users/pablosoriano/Documents/Data Science/bbsr-challenge/csv/pollutant_combinations.csv", delimiter=",", encoding = result["encoding"], low_memory=False)

In [ ]:
# Assesing if to merge the dataframes or just keep the one from 2024

# Compare unique material names between datasets
materials_2020 = set(oko_2020_df["Name (en)"].dropna().unique())
materials_2023 = set(oko_2023_df["Name (en)"].dropna().unique())
materials_2024 = set(oko_2024_df["Name (en)"].dropna().unique())

# Find materials in 2020 and 2023 that are not in 2024
unique_2020 = materials_2020 - materials_2024
unique_2023 = materials_2023 - materials_2024

# Count them
unique_2020_count = len(unique_2020)
unique_2023_count = len(unique_2023)

unique_2020_count, unique_2023_count


(262, 546)

There are materials in previous datasets that are not present in the 2024 dataset:

2020 has 262 materials not found in 2024.
2023 has 546 materials not found in 2024.

This means that the 2024 dataset is not a full replacement — it doesn’t include all materials from earlier versions.

For maximum coverage (e.g. all materials ever documented), we should consider including previous years.

In [21]:
# merging the datasets

# Add a source column for reference
oko_2020_df["source_year"] = 2020
oko_2023_df["source_year"] = 2023
oko_2024_df["source_year"] = 2024

# Combine all datasets
oko_combined_df = pd.concat([oko_2020_df, oko_2023_df, oko_2024_df], ignore_index=True)

In [22]:
# Sort by source year descending to keep the latest entry in case of duplicates
combined_df_sorted = oko_combined_df.sort_values(by="source_year", ascending=False)


In [23]:
# Drop duplicates keeping the latest version (based on UUID, Version, Modul)
deduped_df = combined_df_sorted.drop_duplicates(subset=["UUID", "Version", "Modul"], keep="first")

# Reset index and show result
deduped_df.reset_index(drop=True, inplace=True)

In [ ]:
deduped_df

## Data Cleaning

In [24]:
# Identifying the columns with missing values
missing_values = deduped_df.isnull().sum()

In [25]:
missing_values

UUID                        0
Version                     0
Name (de)                2886
Name (en)                2953
Kategorie (original)        0
                        ...  
WDP (A2)                 9405
Unnamed: 79             25665
source_year                 0
Stueckgewicht (kg)      25477
Unnamed: 80             25665
Length: 83, dtype: int64

In [27]:
# Calculate percentage of missing values per column
null_percentage = deduped_df.isnull().mean().sort_values(ascending=False)

# Select columns with more than 90% missing values
high_null_cols = null_percentage[null_percentage > 0.9]
high_null_cols

URL des Vorgängers        1.000000
Unnamed: 80               1.000000
Unnamed: 79               1.000000
Stueckgewicht (kg)        0.992675
Ergiebigkeit (m2)         0.987064
Schuettdichte (kg/m3)     0.986090
Laengengewicht (kg/m)     0.941126
Version des Vorgängers    0.937502
dtype: float64

These columns have extremely low data coverage and are unlikely to offer analytical value without substantial data imputation or external sourcing. Dropping them will simplify the dataset and reduce noise without significant information loss.

Unnamed: 80 and URL des Vorgängers are completely empty → should be dropped.


In [28]:
# Drop the columns with >90% missing values
columns_to_drop = [
    "Unnamed: 80", "Unnamed: 79", "URL des Vorgängers",
    "Stueckgewicht (kg)", "Ergiebigkeit (m2)", "Schuettdichte (kg/m3)",
    "Laengengewicht (kg/m)", "Version des Vorgängers"
]

cleaned_df = deduped_df.drop(columns=columns_to_drop)

Missing values of Environmental Impact Indicators (EN 15804+A1 and +2)

Understanding the EN 15804 Standard and Its Impact on LCA Calculations
The EN 15804 standard is the foundation for Environmental Product Declarations (EPDs) worldwide. In 2019, a significant update (EN 15804+A2) was released, becoming mandatory in July 2022. This update changes how environmental impacts are measured in EPDs.

What Changed?
While the environmental impact categories (like global warming and ozone depletion) are mostly the same, the units used to measure them have changed. This means that older EPDs (EN 15804+A1) cannot be directly compared or used with newer EPDs (EN 15804+A2) in LCA calculations.

How does this affect Building LCA?
Simply put, you can't mix and match old and new EPDs in your building LCA calculations.

Since 15804+A1 has a big % of missing values and it's out of use, I will evaluate drop the whole columns unless they have values that are not covered in +A2



In [29]:
# Define A1 and A2 column groups
a1_columns = ["GWP", "ODP", "POCP", "AP", "EP", "ADPE", "ADPF"]
a2_columns = [
    "GWPtotal (A2)", "GWPbiogenic (A2)", "GWPfossil (A2)", "GWPluluc (A2)",
    "ODP (A2)", "POCP (A2)", "AP (A2)", "EPmarine (A2)", "EPfreshwater (A2)", 
    "EPterrestrial (A2)", "ADPE (A2)", "ADPF (A2)"
]

# Check for non-null rows in A1 indicators
a1_non_null = (cleaned_df[a1_columns].notnull().sum()).sort_values(ascending=False)

# Check for any A1 values that do not have A2 counterparts in the same row
# Simplest heuristic: check if A1 column is filled but all A2s are null for that row
rows_with_only_a1 = cleaned_df[a1_columns].notnull().any(axis=1) & (~cleaned_df[a2_columns].notnull().any(axis=1))

# Count how many rows this applies to
only_a1_count = rows_with_only_a1.sum()

a1_non_null, only_a1_count


(GWP     8294
 ODP     8281
 POCP    8281
 AP      8281
 ADPE    8281
 EP      8279
 ADPF    8277
 dtype: int64,
 np.int64(8294))

Interpretation: These A1 values are not duplicated or represented in the A2 columns — they are unique to older datasets.
Therefore, dropping them would result in data loss for ~8,300 records.

## Data Transformation

In [31]:
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25665 entries, 0 to 25664
Data columns (total 76 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   UUID                                           25665 non-null  object 
 1   Version                                        25665 non-null  object 
 2   Name (de)                                      22779 non-null  object 
 3   Name (en)                                      22712 non-null  object 
 4   Kategorie (original)                           25665 non-null  object 
 5   Kategorie (en)                                 25665 non-null  object 
 6   Konformität                                    25665 non-null  object 
 7   Laenderkennung                                 25659 non-null  object 
 8   Typ                                            25665 non-null  object 
 9   Referenzjahr                                   256

In [ ]:
# Convert numeric columns (A2 environmental indicators and other relevant ones) from string to float
# We'll replace commas with dots and coerce errors to NaN

numeric_cols = [
    "Referenzjahr", "Gueltig bis", "GWP", "ODP", "POCP", "AP", "EP", "ADPE", "ADPF","PERE", "PERM", "PERT", "PENRE", "PENRM", "PENRT",
    "SM", "RSF", "NRSF", "FW", "HWD", "NHWD", "RWD", "CRU", "MFR", "MER", "EEE", "EET",
    "AP (A2)", "GWPtotal (A2)", "GWPbiogenic (A2)", "GWPfossil (A2)", "GWPluluc (A2)",
    "ETPfw (A2)", "PM (A2)", "EPmarine (A2)", "EPfreshwater (A2)", "EPterrestrial (A2)",
    "HTPc (A2)", "HTPnc (A2)", "IRP (A2)", "SOP (A2)", "ODP (A2)", "POCP (A2)",
    "ADPF (A2)", "ADPE (A2)", "WDP (A2)"
]

# Convert all columns listed above 
for col in numeric_cols:
    if col in cleaned_df.columns:
        cleaned_df[col] = cleaned_df[col].astype(str).str.replace(",", ".").str.strip()
        cleaned_df[col] = pd.to_numeric(cleaned_df[col], errors='coerce')

# Convert date column to datetime
if "Veroeffentlicht am" in cleaned_df.columns:
    cleaned_df["Veroeffentlicht am"] = pd.to_datetime(cleaned_df["Veroeffentlicht am"], errors='coerce')

# Convert Referenzjahr and Gueltig bis to integer 
cleaned_df["Referenzjahr"] = cleaned_df["Referenzjahr"].dropna().astype("Int64")
cleaned_df["Gueltig bis"] = cleaned_df["Gueltig bis"].dropna().astype("Int64")

- Numeric columns were converted from string (with commas) to proper float types.

- Date column Veroeffentlicht am is now in datetime format.

- Year columns like Referenzjahr and Gueltig bis were converted to integer type where possible.

In [ ]:
cleaned_df

#REVIEW - Was thinking of imputing missing values of A2 columns with the median for category and module, but two materials in the same Kategorie (en) and Modul can differ a lot in their environmental impact or circularity, especially depending on:

- Production method (e.g., recycled vs virgin)
- Geographic source (local vs imported)
- Manufacturer-specific practices
- Declared unit or functional equivalence

Let's check if we can impute this values from other sources

## Data Enrichment

In [30]:
# Helper column classifying the environmental impact standard
def classify_impact_standard(row):
    has_a1 = row[a1_columns].notnull().any()
    has_a2 = row[a2_columns].notnull().any()
    if has_a1 and not has_a2:
        return "A1"
    elif has_a2 and not has_a1:
        return "A2"
    elif has_a1 and has_a2:
        return "mixed"
    else:
        return "none"

# Apply the function
cleaned_df["impact_standard"] = cleaned_df.apply(classify_impact_standard, axis=1)

# Show counts for each category
impact_counts = cleaned_df["impact_standard"].value_counts()

impact_counts


impact_standard
A2      16260
A1       8294
none     1111
Name: count, dtype: int64

### Grouping high cardinality categorical variables

Why Group High-Cardinality Categorical Variables?

- Simplifies the Data
- Some text columns may have hundreds of unique values but little analytical meaning (e.g., slight naming variations).
- Improves Interpretability
- Creating higher-level groupings makes it easier to see patterns (e.g., grouping brands, regions, or dataset types).
- Reduces Noise in Visualizations/Models
- Too many categories can overwhelm bar charts or lead to sparse dummy variables in ML models.

In [34]:
# Filter the DataFrame for object columns
non_numeric = cleaned_df.select_dtypes("object")

# Loop through columns
for x in non_numeric.columns:
    # Print the number of unique values
    print(f"Number of unique values in {x} column: ", non_numeric[x].nunique())

cleaned_df

Number of unique values in UUID column:  3863
Number of unique values in Version column:  118
Number of unique values in Name (de) column:  2459
Number of unique values in Name (en) column:  2495
Number of unique values in Kategorie (original) column:  264
Number of unique values in Kategorie (en) column:  265
Number of unique values in Konformität column:  16
Number of unique values in Laenderkennung column:  42
Number of unique values in Typ column:  5
Number of unique values in URL column:  3984
Number of unique values in Declaration owner column:  439
Number of unique values in Registrierungsnummer column:  1567
Number of unique values in Registrierungsstelle column:  11
Number of unique values in UUID des Vorgängers column:  809
Number of unique values in Bezugsgroesse column:  37
Number of unique values in Bezugseinheit column:  9
Number of unique values in Referenzfluss-UUID column:  2549
Number of unique values in Referenzfluss-Name column:  2280
Number of unique values in Flae

,UUID,Version,Name (de),Name (en),Kategorie (original),Kategorie (en),Konformität,Laenderkennung,Typ,Referenzjahr,...,HTPnc (A2),IRP (A2),SOP (A2),ODP (A2),POCP (A2),ADPF (A2),ADPE (A2),WDP (A2),source_year,impact_standard
0,3501e11c-2909-4e3f-a976-2fa82c86e803,00.00.009,Claytec Mineral 20 trocken,Claytec Mineral 20 dried,'Mineralische Baustoffe' / 'Mörtel und Beton' ...,'Mineral building products' / 'Mortar and Conc...,'EN 15804+A2 (EF 3.0)' / 'PCR Ökobilanzierung ...,DE,specific dataset,2022,...,NaN,NaN,NaN,-2.549346e-06,-2.049393e-05,-0.462753,-4.342871e-07,-0.000162,2024,A2
1,00fe2655-02ad-4ded-925a-af9b8f1a27c8,00.00.015,FASBA e.V. Baustroh 100 kg/m³,FASBA e.V. Baustroh 100 kg/m³ GaBi - 2024,'Dämmstoffe' / 'Stroh' / 'Strohballen','Insulation materials' / 'Straw' / 'Straw bale','EN 15804+A2 (EF 3.1)' / 'ISO 14025',DE,specific dataset,2024,...,-1.931275e-07,-0.164974,-11.855288,-3.539990e-11,-1.033700e-02,-441.423176,-8.462049e-07,-0.055600,2024,A2
2,00fe2655-02ad-4ded-925a-af9b8f1a27c8,00.00.015,FASBA e.V. Baustroh 100 kg/m³,FASBA e.V. Baustroh 100 kg/m³ GaBi - 2024,'Dämmstoffe' / 'Stroh' / 'Strohballen','Insulation materials' / 'Straw' / 'Straw bale','EN 15804+A2 (EF 3.1)' / 'ISO 14025',DE,specific dataset,2024,...,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,2024,A2
3,00fe2655-02ad-4ded-925a-af9b8f1a27c8,00.00.015,FASBA e.V. Baustroh 100 kg/m³,FASBA e.V. Baustroh 100 kg/m³ GaBi - 2024,'Dämmstoffe' / 'Stroh' / 'Strohballen','Insulation materials' / 'Straw' / 'Straw bale','EN 15804+A2 (EF 3.1)' / 'ISO 14025',DE,specific dataset,2024,...,2.743039e-08,0.238850,16.740538,5.242030e-11,2.433899e-02,24.347930,3.594269e-07,0.219631,2024,A2
4,3501e11c-2909-4e3f-a976-2fa82c86e803,00.00.009,Claytec Mineral 20 trocken,Claytec Mineral 20 dried,'Mineralische Baustoffe' / 'Mörtel und Beton' ...,'Mineral building products' / 'Mortar and Conc...,'EN 15804+A2 (EF 3.0)' / 'PCR Ökobilanzierung ...,DE,specific dataset,2022,...,NaN,NaN,NaN,8.308000e-05,3.862000e-11,0.199900,6.946000e-07,0.001645,2024,A2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25660,34f8d4b9-2ff3-4ca3-aab0-d52c6e455f71,00.00.013,Isospan 1 GaBi Holzbetonmantelsteine mit Holzf...,NaN,'Mineralische Baustoffe' / 'Steine und Element...,"'Mineral building products' / 'Bricks, blocks ...",'EN 15804' / 'ISO 14025',AT,specific dataset,2017,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020,A1
25661,34f8d4b9-2ff3-4ca3-aab0-d52c6e455f71,00.00.013,Isospan 1 GaBi Holzbetonmantelsteine mit Holzf...,NaN,'Mineralische Baustoffe' / 'Steine und Element...,"'Mineral building products' / 'Bricks, blocks ...",'EN 15804' / 'ISO 14025',AT,specific dataset,2017,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020,A1
25662,8d06b1df-e898-4009-adee-57ca44aaafcc,00.05.000,BauderTHERMOFOL,BauderTHERMOFOL U / M,'Kunststoffe' / 'Dachbahnen' / 'PVC-Dachbahnen','Plastics' / 'Roofing membranes' / 'PVC sheet','DIN EN 15804' / 'ISO 14025',DE,average dataset,2016,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020,A1
25663,d4680073-0c9c-40f4-842f-c6830856812c,00.03.000,Betonpflaster- Standardstein grau mit Vorsatz,NaN,'Mineralische Baustoffe' / 'Steine und Element...,"'Mineral building products' / 'Bricks, blocks ...",'DIN EN 15804' / 'ISO 14025',DE,average dataset,2015,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020,A1


## Data Validation

# EDA

# Machine Learning